In [1]:
from app.loader import load_docs
from app.splitter import TextSplitter, token_size

In [2]:
docs = load_docs()

100%|██████████| 12/12 [00:45<00:00,  3.75s/it]

Loaded 11 PDF documents


In [3]:
chunks = []
text_splitter = TextSplitter(chunk_size=512)

for i, doc in enumerate(docs):
    doc_chunks = text_splitter(doc)
    chunks += doc_chunks
    print(f'Doc {i+1}: {token_size(doc)} tokens, {len(doc_chunks)} chunks')

print('\nTotal chunks', len(chunks))


Doc 1: 45366 tokens, 91 chunks
Doc 2: 65671 tokens, 131 chunks
Doc 3: 25519 tokens, 51 chunks
Doc 4: 46689 tokens, 94 chunks
Doc 5: 48980 tokens, 98 chunks
Doc 6: 61540 tokens, 126 chunks
Doc 7: 24178 tokens, 49 chunks
Doc 8: 50730 tokens, 102 chunks
Doc 9: 41999 tokens, 84 chunks
Doc 10: 46333 tokens, 93 chunks
Doc 11: 50055 tokens, 101 chunks

Total chunks 1020
